In [ ]:
WITH bb_rest as (
  SELECT product_line_id,market_place_listing,purchase_price,in_stock from `gcp-takealot.prod_user_tracking.impression_action` as bb_rest,
  UNNEST(bb_rest.payload.data.products) p
  WHERE 
     (bb_rest.payload.event_timestamp) BETWEEN TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -2 hour) AND CURRENT_TIMESTAMP()
     AND (bb_rest.timestamp) BETWEEN TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -2 hour) AND CURRENT_TIMESTAMP()
     AND payload.context = "product_details.offers.new"
)

SELECT
       DISTINCT bb_winner.payload.data.product.product_line_id as PLID,
       CASE
         WHEN bb_winner.payload.data.product.market_place_listing = true THEN "Marketplace"
         ELSE "Retail"
      END AS buybox_winner,
      bb_winner.payload.data.product.purchase_price as buybox_winner_price,
      bb_rest.purchase_price AS Retail_Price,
      DATE(FORMAT_DATE("%Y-%m-%d",DATETIME(bb_winner.timestamp,'Africa/Johannesburg'))) AS Date,
      CASE
         WHEN bb_winner.payload.data.product.in_stock=true THEN "Instock"
         ELSE "LeadTime"
      END AS Marketplace_Instock,
      CASE
          WHEN bb_rest.in_stock=true THEN "Instock"
          ELSE "LeadTime"
      END AS Retail_Instock,
    bh.bh_division AS Division, bh.bh_reporting_department AS Department, bh.bh_level_1 AS Level1,bh.tsin_title , bh.brand_name AS BrandName
  FROM `gcp-takealot.prod_user_tracking.impression_action` AS bb_winner
        LEFT JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bb_winner.payload.data.product.product_line_id = bh.product_line_id
        JOIN bb_rest ON bb_rest.product_line_id = bh.product_line_id
  WHERE
     (bb_winner.payload.event_timestamp) BETWEEN TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -2 hour) AND CURRENT_TIMESTAMP()
     AND (bb_winner.timestamp) BETWEEN TIMESTAMP_ADD(CURRENT_TIMESTAMP(), INTERVAL -2 hour) AND CURRENT_TIMESTAMP()
     
    AND bb_rest.product_line_id = bb_winner.payload.data.product.product_line_id
    AND bb_winner.payload.context = "product_details"
    AND bb_winner.payload.data.product.market_place_listing IS NOT NULL
    AND bb_winner.payload.data.product.sku_id>0
    AND bb_winner.payload.data.product.market_place_listing IN (true)
    AND bb_winner.payload.data.product.market_place_listing <> bb_rest.market_place_listing